In [1]:
import random
import copy

import numpy as np
from numpy.linalg import norm

In [2]:
class State:
    def __init__(self, route: [], distance: int = 0):
        self.route = route
        self.distance = distance

    def __eq__(self, other):
        for i in range(len(self.route)):
            if self.route[i] != other.route[i]:
                return False
        return True

    def __lt__(self, other):
        return self.distance < other.distance

    def __repr__(self):
        return "({0},{1})\n".format(self.route, self.distance)

    def copy(self):
        return State(self.route, self.distance)

    def deepcopy(self):
        return State(copy.deepcopy(self.route), copy.deepcopy(self.distance))

    def update_distance(self, matrix, home):
        self.distance = 0
        from_index = home
        for i in range(len(self.route)):
            self.distance += matrix[from_index][self.route[i]]
            from_index = self.route[i]
        self.distance += matrix[from_index][home]




In [4]:
class City:
    def __init__(self, index: int, distance: int):
        self.index = index
        self.distance = distance

    def __lt__(self, other):
        return self.distance < other.distance


def get_random_solution(
    matrix: [], home: int, city_indexes: [], size: int
):
    cities = city_indexes.copy()
    cities.pop(home)
    population = []
    for i in range(size):
        random.shuffle(cities)
        state = State(cities[:])
        state.update_distance(matrix, home)
        population.append(state)
    population.sort()
    return population[0]


def mutate(matrix: [], home: int, state: State, mutation_rate: float = 0.01):
    mutated_state = state.deepcopy()
    for i in range(len(mutated_state.route)):
        if random.random() < mutation_rate:
            j = int(random.random() * len(state.route))
            city_1 = mutated_state.route[i]
            city_2 = mutated_state.route[j]
            mutated_state.route[i] = city_2
            mutated_state.route[j] = city_1
    mutated_state.update_distance(matrix, home)
    return mutated_state


def hill_climbing(
    matrix: [],
    home: int,
    initial_state: State,
    max_iterations: int,
    mutation_rate: float = 0.01,
):
    best_state = initial_state
    iterator = 0
    while True:
        neighbor = mutate(matrix, home, best_state, mutation_rate)
        if neighbor.distance >= best_state.distance:
            iterator += 1
            if iterator > max_iterations:
                break
        if neighbor.distance < best_state.distance:
            best_state = neighbor
    return best_state


def get_euclidean_distance(p, q):
    return round(norm(np.array(p) - np.array(q)))


def main():

 # Extracted from the Western Sahara city coordinates
    # found in http://www.math.uwaterloo.ca/tsp/world/countries.html
    cities_coordinates = {
        1:  (20833.3333, 17100.0000),
        2:  (20900.0000, 17066.6667),
        3:  (21300.0000, 13016.6667),
        4:  (21600.0000, 14150.0000),
        5:  (21600.0000, 14966.6667),
        6:  (21600.0000, 16500.0000),
        7:  (22183.3333, 13133.3333),
        8:  (22583.3333, 14300.0000),
        9:  (22683.3333, 12716.6667),
        10: (23616.6667, 15866.6667),
        11: (23700.0000, 15933.3333),
        12: (23883.3333, 14533.3333),
        13: (24166.6667, 13250.0000),
        14: (25149.1667, 12365.8333),
        15: (26133.3333, 14500.0000),
        16: (26150.0000, 10550.0000),
        17: (26283.3333, 12766.6667),
        18: (26433.3333, 13433.3333),
        19: (26550.0000, 13850.0000),
        20: (26733.3333, 11683.3333),
        21: (27026.1111, 13051.9444),
        22: (27096.1111, 13415.8333),
        23: (27153.6111, 13203.3333),
        24: (27166.6667, 9833.3333),
        25: (27233.3333, 10450.0000),
        26: (27233.3333, 11783.3333),
        27: (27266.6667, 10383.3333),
        28: (27433.3333, 12400.0000),
        29: (27462.5000, 12992.2222),
    }

    D = []
    for _, target_coordinates in cities_coordinates.items():
        distances = []
        for _, coordinates in cities_coordinates.copy().items():
            distances.append(get_euclidean_distance(target_coordinates, coordinates))
        D.append(distances)
    
    n = random.randint(0,29)

    home = n
    max_iterations = 1000000
    cities = list(cities_coordinates.keys())
    city_indexes = [index - 1 for index in cities]

    state = get_random_solution(D, home, city_indexes, 100)
    print("-- Initial state solution --")
    print(cities[home], end="")
    for i in range(0, len(state.route)):
        print(" -> " + str(cities[state.route[i]]), end="")
    print(" -> " + str(cities[home]), end="")
    print("\n\nTotal distance: {0} miles".format(state.distance))
    print()

    state = hill_climbing(D, home, state, max_iterations, 0.1)
    print("-- Hill climbing solution --")
    print(cities[home], end="")
    for i in range(0, len(state.route)):
        print(" -> " + str(cities[state.route[i]]), end="")
    print(" -> " + str(cities[home]), end="")
    print("\n\nTotal distance: {0} miles".format(state.distance))
    print()


if __name__ == "__main__":
    main()

-- Initial state solution --
14 -> 11 -> 25 -> 27 -> 22 -> 17 -> 13 -> 21 -> 23 -> 19 -> 29 -> 28 -> 15 -> 8 -> 24 -> 9 -> 4 -> 7 -> 1 -> 5 -> 18 -> 20 -> 16 -> 12 -> 6 -> 2 -> 3 -> 10 -> 26 -> 14

Total distance: 81655 miles

-- Hill climbing solution --
fitness obtained: 34563
14 -> 13 -> 12 -> 10 -> 11 -> 15 -> 19 -> 18 -> 22 -> 23 -> 21 -> 29 -> 28 -> 26 -> 20 -> 17 -> 8 -> 6 -> 2 -> 1 -> 5 -> 4 -> 3 -> 7 -> 9 -> 16 -> 24 -> 27 -> 25 -> 14

Total distance: 34563 miles



In [ ]:
class City:
    def __init__(self, index: int, distance: int):
        self.index = index
        self.distance = distance

    def __lt__(self, other):
        return self.distance < other.distance


def get_random_solution(
    matrix: [], home: int, city_indexes: [], size: int
):
    cities = city_indexes.copy()
    cities.pop(home)
    population = []
    for i in range(size):
        random.shuffle(cities)
        state = State(cities[:])
        state.update_distance(matrix, home)
        population.append(state)
    population.sort()
    return population[0]


def mutate(matrix: [], home: int, state: State, mutation_rate: float = 0.01):
    mutated_state = state.deepcopy()
    for i in range(len(mutated_state.route)):
        if random.random() < mutation_rate:
            j = int(random.random() * len(state.route))
            city_1 = mutated_state.route[i]
            city_2 = mutated_state.route[j]
            mutated_state.route[i] = city_2
            mutated_state.route[j] = city_1
    mutated_state.update_distance(matrix, home)
    return mutated_state


def hill_climbing(
    matrix: [],
    home: int,
    initial_state: State,
    max_iterations: int,
    mutation_rate: float = 0.01,
):
    best_state = initial_state
    iterator = 0
    while True:
        neighbor = mutate(matrix, home, best_state, mutation_rate)
        if neighbor.distance >= best_state.distance:
            iterator += 1
            if iterator > max_iterations:
                break
        if neighbor.distance < best_state.distance:
            best_state = neighbor
    return best_state


def get_euclidean_distance(p, q):
    return round(norm(np.array(p) - np.array(q)))


def main():

 # Extracted from the Djibouti city coordinates
    # found in http://www.math.uwaterloo.ca/tsp/world/countries.html
    cities_coordinates = {
        1	:		(	11003.6111	,	42102.5	),
        2	:		(	11108.6111	,	42373.8889	),
        3	:		(	11133.3333	,	42885.8333	),
        4	:		(	11155.8333	,	42712.5	),
        5	:		(	11183.3333	,	42933.3333	),
        6	:		(	11297.5	,	42853.3333	),
        7	:		(	11310.2778	,	42929.4444	),
        8	:		(	11416.6667	,	42983.3333	),
        9	:		(	11423.8889	,	43000.2778	),
        10	:		(	11438.3333	,	42057.2222	),
        11	:		(	11461.1111	,	43252.7778	),
        12	:		(	11485.5556	,	43187.2222	),
        13	:		(	11503.0556	,	42855.2778	),
        14	:		(	11511.3889	,	42106.3889	),
        15	:		(	11522.2222	,	42841.9444	),
        16	:		(	11569.4444	,	43136.6667	),
        17	:		(	11583.3333	,	43150	),
        18	:		(	11595	,	43148.0556	),
        19	:		(	11600	,	43150	),
        20	:		(	11690.5556	,	42686.6667	),
        21	:		(	11715.8333	,	41836.1111	),
        22	:		(	11751.1111	,	42814.4444	),
        23	:		(	11770.2778	,	42651.9444	),
        24	:		(	11785.2778	,	42884.4444	),
        25	:		(	11822.7778	,	42673.6111	),
        26	:		(	11846.9444	,	42660.5556	),
        27	:		(	11963.0556	,	43290.5556	),
        28	:		(	11973.0556	,	43026.1111	),
        29	:		(	12058.3333	,	42195.5556	),
        30	:		(	12149.4444	,	42477.5	),
        31	:		(	12286.9444	,	43355.5556	),
        32	:		(	12300	,	42433.3333	),
        33	:		(	12355.8333	,	43156.3889	),
        34	:		(	12363.3333	,	43189.1667	),
        35	:		(	12372.7778	,	42711.3889	),
        36	:		(	12386.6667	,	43334.7222	),
        37	:		(	12421.6667	,	42895.5556	),
        38	:		(	12645	,	42973.3333	),
    }

    D = []
    for _, target_coordinates in cities_coordinates.items():
        distances = []
        for _, coordinates in cities_coordinates.copy().items():
            distances.append(get_euclidean_distance(target_coordinates, coordinates))
        D.append(distances)

    home = 0
    max_iterations = 1000000
    cities = list(cities_coordinates.keys())
    city_indexes = [index - 1 for index in cities]

    state = get_random_solution(D, home, city_indexes, 100)
    print("-- Initial state solution --")
    print(cities[home], end="")
    for i in range(0, len(state.route)):
        print(" -> " + str(cities[state.route[i]]), end="")
    print(" -> " + str(cities[home]), end="")
    print("\n\nTotal distance: {0} miles".format(state.distance))
    print()

    state = hill_climbing(D, home, state, max_iterations, 0.1)
    print("-- Hill climbing solution --")
    print(cities[home], end="")
    for i in range(0, len(state.route)):
        print(" -> " + str(cities[state.route[i]]), end="")
    print(" -> " + str(cities[home]), end="")
    print("\n\nTotal distance: {0} miles".format(state.distance))
    print()


if __name__ == "__main__":
    main()

-- Initial state solution --
1 -> 26 -> 7 -> 38 -> 5 -> 3 -> 24 -> 32 -> 34 -> 19 -> 29 -> 21 -> 16 -> 22 -> 20 -> 14 -> 10 -> 8 -> 36 -> 30 -> 27 -> 18 -> 17 -> 9 -> 28 -> 37 -> 35 -> 23 -> 25 -> 33 -> 12 -> 6 -> 11 -> 2 -> 4 -> 15 -> 31 -> 13 -> 1

Total distance: 24419 miles

-- Hill climbing solution --
1 -> 4 -> 3 -> 5 -> 6 -> 7 -> 27 -> 31 -> 36 -> 34 -> 33 -> 38 -> 37 -> 35 -> 32 -> 30 -> 29 -> 21 -> 10 -> 14 -> 23 -> 26 -> 25 -> 22 -> 24 -> 28 -> 19 -> 18 -> 17 -> 16 -> 12 -> 11 -> 9 -> 8 -> 13 -> 15 -> 20 -> 2 -> 1

Total distance: 8303 miles



In [ ]:
class City:
    def __init__(self, index: int, distance: int):
        self.index = index
        self.distance = distance

    def __lt__(self, other):
        return self.distance < other.distance


def get_random_solution(
    matrix: [], home: int, city_indexes: [], size: int
):
    cities = city_indexes.copy()
    cities.pop(home)
    population = []
    for i in range(size):
        random.shuffle(cities)
        state = State(cities[:])
        state.update_distance(matrix, home)
        population.append(state)
    population.sort()
    return population[0]


def mutate(matrix: [], home: int, state: State, mutation_rate: float = 0.01):
    mutated_state = state.deepcopy()
    for i in range(len(mutated_state.route)):
        if random.random() < mutation_rate:
            j = int(random.random() * len(state.route))
            city_1 = mutated_state.route[i]
            city_2 = mutated_state.route[j]
            mutated_state.route[i] = city_2
            mutated_state.route[j] = city_1
    mutated_state.update_distance(matrix, home)
    return mutated_state


def hill_climbing(
    matrix: [],
    home: int,
    initial_state: State,
    max_iterations: int,
    mutation_rate: float = 0.01,
):
    best_state = initial_state
    iterator = 0
    while True:
        neighbor = mutate(matrix, home, best_state, mutation_rate)
        if neighbor.distance >= best_state.distance:
            iterator += 1
            if iterator > max_iterations:
                break
        if neighbor.distance < best_state.distance:
            best_state = neighbor
    return best_state


def get_euclidean_distance(p, q):
    return round(norm(np.array(p) - np.array(q)))


def main():

 # Extracted from the Qatar city coordinates
    # found in http://www.math.uwaterloo.ca/tsp/world/countries.html
    cities_coordinates = {
        1	:		(	24748.3333	,	50840	),
        2	:		(	24758.8889	,	51211.9444	),
        3	:		(	24827.2222	,	51394.7222	),
        4	:		(	24904.4444	,	51175	),
        5	:		(	24996.1111	,	51548.8889	),
        6	:		(	25010	,	51039.4444	),
        7	:		(	25030.8333	,	51275.2778	),
        8	:		(	25067.7778	,	51077.5	),
        9	:		(	25100	,	51516.6667	),
        10	:		(	25103.3333	,	51521.6667	),
        11	:		(	25121.9444	,	51218.3333	),
        12	:		(	25150.8333	,	51537.7778	),
        13	:		(	25158.3333	,	51163.6111	),
        14	:		(	25162.2222	,	51220.8333	),
        15	:		(	25167.7778	,	51606.9444	),
        16	:		(	25168.8889	,	51086.3889	),
        17	:		(	25173.8889	,	51269.4444	),
        18	:		(	25210.8333	,	51394.1667	),
        19	:		(	25211.3889	,	51619.1667	),
        20	:		(	25214.1667	,	50807.2222	),
        21	:		(	25214.4444	,	51378.8889	),
        22	:		(	25223.3333	,	51451.6667	),
        23	:		(	25224.1667	,	51174.4444	),
        24	:		(	25233.3333	,	51333.3333	),
        25	:		(	25234.1667	,	51203.0556	),
        26	:		(	25235.5556	,	51330	),
        27	:		(	25235.5556	,	51495.5556	),
        28	:		(	25242.7778	,	51428.8889	),
        29	:		(	25243.0556	,	51452.5	),
        30	:		(	25252.5	,	51559.1667	),
        31	:		(	25253.8889	,	51535.2778	),
        32	:		(	25253.8889	,	51549.7222	),
        33	:		(	25256.9444	,	51398.8889	),
        34	:		(	25263.6111	,	51516.3889	),
        35	:		(	25265.8333	,	51545.2778	),
        36	:		(	25266.6667	,	50969.1667	),
        37	:		(	25266.6667	,	51483.3333	),
        38	:		(	25270.5556	,	51532.7778	),
        39	:		(	25270.8333	,	51505.8333	),
        40	:		(	25270.8333	,	51523.0556	),
        41	:		(	25275.8333	,	51533.6111	),
        42	:		(	25277.2222	,	51547.7778	),
        43	:		(	25278.3333	,	51525.5556	),
        44	:		(	25278.3333	,	51541.3889	),
        45	:		(	25279.1667	,	51445.5556	),
        46	:		(	25281.1111	,	51535	),
        47	:		(	25281.3889	,	51512.5	),
        48	:		(	25283.3333	,	51533.3333	),
        49	:		(	25283.6111	,	51546.6667	),
        50	:		(	25284.7222	,	51555.2778	),
        51	:		(	25286.1111	,	51504.1667	),
        52	:		(	25286.1111	,	51534.1667	),
        53	:		(	25286.6667	,	51533.3333	),
        54	:		(	25287.5	,	51537.7778	),
        55	:		(	25288.0556	,	51546.6667	),
        56	:		(	25290.8333	,	51528.3333	),
        57	:		(	25291.9444	,	51424.4444	),
        58	:		(	25292.5	,	51520.8333	),
        59	:		(	25298.6111	,	51001.6667	),
        60	:		(	25300.8333	,	51394.4444	),
        61	:		(	25306.9444	,	51507.7778	),
        62	:		(	25311.9444	,	51003.0556	),
        63	:		(	25313.8889	,	50883.3333	),
        64	:		(	25315.2778	,	51438.6111	),
        65	:		(	25316.6667	,	50766.6667	),
        66	:		(	25320.5556	,	51495.5556	),
        67	:		(	25322.5	,	51507.7778	),
        68	:		(	25325.2778	,	51470	),
        69	:		(	25326.6667	,	51350.2778	),
        70	:		(	25337.5	,	51425	),
        71	:		(	25339.1667	,	51173.3333	),
        72	:		(	25340.5556	,	51293.6111	),
        73	:		(	25341.9444	,	51507.5	),
        74	:		(	25358.8889	,	51333.6111	),
        75	:		(	25363.6111	,	51281.1111	),
        76	:		(	25368.6111	,	51226.3889	),
        77	:		(	25374.4444	,	51436.6667	),
        78	:		(	25377.7778	,	51294.7222	),
        79	:		(	25396.9444	,	51422.5	),
        80	:		(	25400	,	51183.3333	),
        81	:		(	25400	,	51425	),
        82	:		(	25404.7222	,	51073.0556	),
        83	:		(	25416.9444	,	51403.8889	),
        84	:		(	25416.9444	,	51457.7778	),
        85	:		(	25419.4444	,	50793.6111	),
        86	:		(	25429.7222	,	50785.8333	),
        87	:		(	25433.3333	,	51220	),
        88	:		(	25440.8333	,	51378.0556	),
        89	:		(	25444.4444	,	50958.3333	),
        90	:		(	25451.3889	,	50925	),
        91	:		(	25459.1667	,	51316.6667	),
        92	:		(	25469.7222	,	51397.5	),
        93	:		(	25478.0556	,	51362.5	),
        94	:		(	25480.5556	,	50938.8889	),
        95	:		(	25483.3333	,	51383.3333	),
        96	:		(	25490.5556	,	51373.6111	),
        97	:		(	25492.2222	,	51400.2778	),
        98	:		(	25495	,	50846.6667	),
        99	:		(	25495	,	50965.2778	),
        100	:		(	25497.5	,	51485.2778	),
        101	:		(	25500.8333	,	50980.5556	),
        102	:		(	25510.5556	,	51242.2222	),
        103	:		(	25531.9444	,	51304.4444	),
        104	:		(	25533.3333	,	50977.2222	),
        105	:		(	25538.8889	,	51408.3333	),
        106	:		(	25545.8333	,	51387.5	),
        107	:		(	25549.7222	,	51431.9444	),
        108	:		(	25550	,	51433.3333	),
        109	:		(	25560.2778	,	51158.6111	),
        110	:		(	25566.9444	,	51484.7222	),
        111	:		(	25567.5	,	50958.8889	),
        112	:		(	25574.7222	,	51486.3889	),
        113	:		(	25585.5556	,	51151.3889	),
        114	:		(	25609.4444	,	51092.2222	),
        115	:		(	25610.2778	,	51475.2778	),
        116	:		(	25622.5	,	51454.4444	),
        117	:		(	25645.8333	,	51450	),
        118	:		(	25650	,	51372.2222	),
        119	:		(	25666.9444	,	51174.4444	),
        120	:		(	25683.8889	,	51505.8333	),
        121	:		(	25686.3889	,	51468.8889	),
        122	:		(	25696.1111	,	51260.8333	),
        123	:		(	25700.8333	,	51584.7222	),
        124	:		(	25708.3333	,	51591.6667	),
        125	:		(	25716.6667	,	51050	),
        126	:		(	25717.5	,	51057.7778	),
        127	:		(	25723.0556	,	51004.1667	),
        128	:		(	25734.7222	,	51547.5	),
        129	:		(	25751.1111	,	51449.1667	),
        130	:		(	25751.9444	,	50920.8333	),
        131	:		(	25758.3333	,	51395.8333	),
        132	:		(	25765.2778	,	51019.7222	),
        133	:		(	25772.2222	,	51483.3333	),
        134	:		(	25775.8333	,	51023.0556	),
        135	:		(	25779.1667	,	51449.7222	),
        136	:		(	25793.3333	,	51409.4444	),
        137	:		(	25808.3333	,	51060.5556	),
        138	:		(	25816.6667	,	51133.3333	),
        139	:		(	25823.6111	,	51152.5	),
        140	:		(	25826.6667	,	51043.8889	),
        141	:		(	25829.7222	,	51245.2778	),
        142	:		(	25833.3333	,	51072.2222	),
        143	:		(	25839.1667	,	51465.2778	),
        144	:		(	25847.7778	,	51205.8333	),
        145	:		(	25850	,	51033.3333	),
        146	:		(	25856.6667	,	51083.3333	),
        147	:		(	25857.5	,	51298.8889	),
        148	:		(	25857.5	,	51441.3889	),
        149	:		(	25866.6667	,	51066.6667	),
        150	:		(	25867.7778	,	51205.5556	),
        151	:		(	25871.9444	,	51354.7222	),
        152	:		(	25872.5	,	51258.3333	),
        153	:		(	25880.8333	,	51221.3889	),
        154	:		(	25883.0556	,	51185.2778	),
        155	:		(	25888.0556	,	51386.3889	),
        156	:		(	25900	,	51000	),
        157	:		(	25904.1667	,	51201.6667	),
        158	:		(	25928.3333	,	51337.5	),
        159	:		(	25937.5	,	51313.3333	),
        160	:		(	25944.7222	,	51456.3889	),
        161	:		(	25950	,	51066.6667	),
        162	:		(	25951.6667	,	51349.7222	),
        163	:		(	25957.7778	,	51075.2778	),
        164	:		(	25958.3333	,	51099.4444	),
        165	:		(	25966.6667	,	51283.3333	),
        166	:		(	25983.3333	,	51400	),
        167	:		(	25983.6111	,	51328.0556	),
        168	:		(	26000.2778	,	51294.4444	),
        169	:		(	26008.6111	,	51083.6111	),
        170	:		(	26016.6667	,	51333.3333	),
        171	:		(	26021.6667	,	51366.9444	),
        172	:		(	26033.3333	,	51116.6667	),
        173	:		(	26033.3333	,	51166.6667	),
        174	:		(	26033.6111	,	51163.8889	),
        175	:		(	26033.6111	,	51200.2778	),
        176	:		(	26048.8889	,	51056.9444	),
        177	:		(	26050	,	51250	),
        178	:		(	26050.2778	,	51297.5	),
        179	:		(	26050.5556	,	51135.8333	),
        180	:		(	26055	,	51316.1111	),
        181	:		(	26067.2222	,	51258.6111	),
        182	:		(	26074.7222	,	51083.6111	),
        183	:		(	26076.6667	,	51166.9444	),
        184	:		(	26077.2222	,	51222.2222	),
        185	:		(	26078.0556	,	51361.6667	),
        186	:		(	26083.6111	,	51147.2222	),
        187	:		(	26099.7222	,	51161.1111	),
        188	:		(	26108.0556	,	51244.7222	),
        189	:		(	26116.6667	,	51216.6667	),
        190	:		(	26123.6111	,	51169.1667	),
        191	:		(	26123.6111	,	51222.7778	),
        192	:		(	26133.3333	,	51216.6667	),
        193	:		(	26133.3333	,	51300	),
        194	:		(	26150.2778	,	51108.0556	),
    }

    D = []
    for _, target_coordinates in cities_coordinates.items():
        distances = []
        for _, coordinates in cities_coordinates.copy().items():
            distances.append(get_euclidean_distance(target_coordinates, coordinates))
        D.append(distances)

    home = 0
    max_iterations = 1000000
    cities = list(cities_coordinates.keys())
    city_indexes = [index - 1 for index in cities]

    state = get_random_solution(D, home, city_indexes, 100)
    print("-- Initial state solution --")
    print(cities[home], end="")
    for i in range(0, len(state.route)):
        print(" -> " + str(cities[state.route[i]]), end="")
    print(" -> " + str(cities[home]), end="")
    print("\n\nTotal distance: {0} miles".format(state.distance))
    print()

    state = hill_climbing(D, home, state, max_iterations, 0.1)
    print("-- Hill climbing solution --")
    print(cities[home], end="")
    for i in range(0, len(state.route)):
        print(" -> " + str(cities[state.route[i]]), end="")
    print(" -> " + str(cities[home]), end="")
    print("\n\nTotal distance: {0} miles".format(state.distance))
    print()


if __name__ == "__main__":
    main()

-- Initial state solution --
1 -> 154 -> 174 -> 161 -> 68 -> 34 -> 157 -> 182 -> 184 -> 66 -> 23 -> 11 -> 113 -> 153 -> 62 -> 128 -> 194 -> 155 -> 181 -> 132 -> 98 -> 14 -> 112 -> 58 -> 106 -> 51 -> 21 -> 179 -> 173 -> 147 -> 52 -> 168 -> 162 -> 131 -> 4 -> 42 -> 9 -> 115 -> 3 -> 133 -> 86 -> 61 -> 41 -> 135 -> 7 -> 50 -> 89 -> 49 -> 84 -> 94 -> 191 -> 57 -> 15 -> 137 -> 118 -> 24 -> 44 -> 101 -> 190 -> 54 -> 67 -> 109 -> 53 -> 16 -> 82 -> 163 -> 48 -> 22 -> 45 -> 29 -> 69 -> 102 -> 144 -> 93 -> 99 -> 146 -> 189 -> 151 -> 83 -> 87 -> 17 -> 107 -> 186 -> 183 -> 164 -> 142 -> 97 -> 120 -> 166 -> 33 -> 150 -> 170 -> 134 -> 25 -> 165 -> 31 -> 77 -> 95 -> 148 -> 28 -> 71 -> 126 -> 111 -> 63 -> 35 -> 117 -> 36 -> 96 -> 19 -> 26 -> 156 -> 177 -> 180 -> 78 -> 39 -> 145 -> 185 -> 143 -> 43 -> 37 -> 178 -> 119 -> 110 -> 12 -> 85 -> 140 -> 92 -> 127 -> 108 -> 79 -> 10 -> 6 -> 40 -> 103 -> 139 -> 60 -> 188 -> 192 -> 159 -> 80 -> 91 -> 64 -> 104 -> 138 -> 158 -> 121 -> 160 -> 81 -> 130 -> 129 -> 18

In [ ]:
class City:
    def __init__(self, index: int, distance: int):
        self.index = index
        self.distance = distance

    def __lt__(self, other):
        return self.distance < other.distance


def get_random_solution(
    matrix: [], home: int, city_indexes: [], size: int
):
    cities = city_indexes.copy()
    cities.pop(home)
    population = []
    for i in range(size):
        random.shuffle(cities)
        state = State(cities[:])
        state.update_distance(matrix, home)
        population.append(state)
    population.sort()
    return population[0]


def mutate(matrix: [], home: int, state: State, mutation_rate: float = 0.01):
    mutated_state = state.deepcopy()
    for i in range(len(mutated_state.route)):
        if random.random() < mutation_rate:
            j = int(random.random() * len(state.route))
            city_1 = mutated_state.route[i]
            city_2 = mutated_state.route[j]
            mutated_state.route[i] = city_2
            mutated_state.route[j] = city_1
    mutated_state.update_distance(matrix, home)
    return mutated_state


def hill_climbing(
    matrix: [],
    home: int,
    initial_state: State,
    max_iterations: int,
    mutation_rate: float = 0.01,
):
    best_state = initial_state
    iterator = 0
    while True:
        neighbor = mutate(matrix, home, best_state, mutation_rate)
        if neighbor.distance >= best_state.distance:
            iterator += 1
            if iterator > max_iterations:
                break
        if neighbor.distance < best_state.distance:
            best_state = neighbor
    return best_state


def get_euclidean_distance(p, q):
    return round(norm(np.array(p) - np.array(q)))


def main():

 # Extracted from the Uruguay city coordinates
    # found in http://www.math.uwaterloo.ca/tsp/world/countries.html
    cities_coordinates = {
        1	:		(	30133.3333	,	57633.3333	),
        2	:		(	30166.6667	,	57100	),
        3	:		(	30233.3333	,	57583.3333	),
        4	:		(	30250	,	56850	),
        5	:		(	30250	,	56950	),
        6	:		(	30250	,	57583.3333	),
        7	:		(	30300	,	56966.6667	),
        8	:		(	30316.6667	,	56816.6667	),
        9	:		(	30400	,	56466.6667	),
        10	:		(	30400	,	56783.3333	),
        11	:		(	30433.3333	,	57433.3333	),
        12	:		(	30466.6667	,	56550	),
        13	:		(	30483.3333	,	56516.6667	),
        14	:		(	30500	,	56450	),
        15	:		(	30500	,	56666.6667	),
        16	:		(	30550	,	57866.6667	),
        17	:		(	30566.6667	,	56883.3333	),
        18	:		(	30600	,	57683.3333	),
        19	:		(	30616.6667	,	56900	),
        20	:		(	30633.3333	,	56166.6667	),
        21	:		(	30683.3333	,	57033.3333	),
        22	:		(	30683.3333	,	57516.6667	),
        23	:		(	30716.6667	,	56600	),
        24	:		(	30733.3333	,	56733.3333	),
        25	:		(	30733.3333	,	57316.6667	),
        26	:		(	30750	,	56750	),
        27	:		(	30783.3333	,	57783.3333	),
        28	:		(	30833.3333	,	56750	),
        29	:		(	30866.6667	,	56366.6667	),
        30	:		(	30900	,	55516.6667	),
        31	:		(	30916.6667	,	56300	),
        32	:		(	30933.3333	,	55483.3333	),
        33	:		(	30933.3333	,	55550	),
        34	:		(	30950	,	56650	),
        35	:		(	30966.6667	,	55550	),
        36	:		(	30966.6667	,	57533.3333	),
        37	:		(	31000	,	55683.3333	),
        38	:		(	31000	,	56250	),
        39	:		(	31016.6667	,	56566.6667	),
        40	:		(	31033.3333	,	56600	),
        41	:		(	31033.3333	,	56883.3333	),
        42	:		(	31083.3333	,	56016.6667	),
        43	:		(	31083.3333	,	56516.6667	),
        44	:		(	31083.3333	,	57016.6667	),
        45	:		(	31083.3333	,	57516.6667	),
        46	:		(	31083.3333	,	57600	),
        47	:		(	31083.3333	,	57833.3333	),
        48	:		(	31100	,	55666.6667	),
        49	:		(	31100	,	55983.3333	),
        50	:		(	31100	,	57016.6667	),
        51	:		(	31116.6667	,	56466.6667	),
        52	:		(	31166.6667	,	55750	),
        53	:		(	31166.6667	,	57000	),
        54	:		(	31166.6667	,	57083.3333	),
        55	:		(	31183.3333	,	56283.3333	),
        56	:		(	31183.3333	,	56933.3333	),
        57	:		(	31200	,	55750	),
        58	:		(	31200	,	57233.3333	),
        59	:		(	31216.6667	,	55666.6667	),
        60	:		(	31216.6667	,	56316.6667	),
        61	:		(	31233.3333	,	55633.3333	),
        62	:		(	31233.3333	,	55883.3333	),
        63	:		(	31233.3333	,	57116.6667	),
        64	:		(	31250	,	55333.3333	),
        65	:		(	31250	,	55950	),
        66	:		(	31250	,	56833.3333	),
        67	:		(	31250	,	57166.6667	),
        68	:		(	31250	,	57200	),
        69	:		(	31283.3333	,	56433.3333	),
        70	:		(	31300	,	55866.6667	),
        71	:		(	31300	,	56016.6667	),
        72	:		(	31300	,	57066.6667	),
        73	:		(	31300	,	57100	),
        74	:		(	31300	,	57133.3333	),
        75	:		(	31300	,	57150	),
        76	:		(	31300	,	57200	),
        77	:		(	31300	,	57700	),
        78	:		(	31316.6667	,	55200	),
        79	:		(	31333.3333	,	57800	),
        80	:		(	31333.3333	,	57833.3333	),
        81	:		(	31333.3333	,	57883.3333	),
        82	:		(	31350	,	55383.3333	),
        83	:		(	31350	,	56633.3333	),
        84	:		(	31350	,	57783.3333	),
        85	:		(	31350	,	57800	),
        86	:		(	31350	,	57983.3333	),
        87	:		(	31366.6667	,	55083.3333	),
        88	:		(	31366.6667	,	55666.6667	),
        89	:		(	31366.6667	,	55850	),
        90	:		(	31383.3333	,	55866.6667	),
        91	:		(	31383.3333	,	57966.6667	),
        92	:		(	31400	,	56366.6667	),
        93	:		(	31400	,	57783.3333	),
        94	:		(	31416.6667	,	54916.6667	),
        95	:		(	31416.6667	,	56683.3333	),
        96	:		(	31433.3333	,	55233.3333	),
        97	:		(	31450	,	55416.6667	),
        98	:		(	31450	,	56383.3333	),
        99	:		(	31466.6667	,	55166.6667	),
        100	:		(	31466.6667	,	56833.3333	),
        101	:		(	31483.3333	,	54833.3333	),
        102	:		(	31500	,	54716.6667	),
        103	:		(	31500	,	55166.6667	),
        104	:		(	31500	,	56033.3333	),
        105	:		(	31500	,	57116.6667	),
        106	:		(	31516.6667	,	54966.6667	),
        107	:		(	31516.6667	,	55766.6667	),
        108	:		(	31516.6667	,	55966.6667	),
        109	:		(	31516.6667	,	57533.3333	),
        110	:		(	31516.6667	,	57583.3333	),
        111	:		(	31533.3333	,	55583.3333	),
        112	:		(	31533.3333	,	55833.3333	),
        113	:		(	31533.3333	,	56050	),
        114	:		(	31550	,	55550	),
        115	:		(	31550	,	57950	),
        116	:		(	31566.6667	,	54550	),
        117	:		(	31583.3333	,	55466.6667	),
        118	:		(	31583.3333	,	57616.6667	),
        119	:		(	31600	,	55850	),
        120	:		(	31600	,	56750	),
        121	:		(	31616.6667	,	56816.6667	),
        122	:		(	31633.3333	,	55000	),
        123	:		(	31633.3333	,	56916.6667	),
        124	:		(	31650	,	56833.3333	),
        125	:		(	31666.6667	,	56166.6667	),
        126	:		(	31666.6667	,	56633.3333	),
        127	:		(	31666.6667	,	57916.6667	),
        128	:		(	31683.3333	,	54650	),
        129	:		(	31683.3333	,	54766.6667	),
        130	:		(	31683.3333	,	55433.3333	),
        131	:		(	31683.3333	,	55950	),
        132	:		(	31683.3333	,	56133.3333	),
        133	:		(	31683.3333	,	57683.3333	),
        134	:		(	31700	,	56100	),
        135	:		(	31716.6667	,	55333.3333	),
        136	:		(	31733.3333	,	54416.6667	),
        137	:		(	31733.3333	,	55983.3333	),
        138	:		(	31750	,	55016.6667	),
        139	:		(	31750	,	55466.6667	),
        140	:		(	31750	,	56066.6667	),
        141	:		(	31766.6667	,	54800	),
        142	:		(	31766.6667	,	56016.6667	),
        143	:		(	31766.6667	,	56250	),
        144	:		(	31783.3333	,	54333.3333	),
        145	:		(	31783.3333	,	54750	),
        146	:		(	31783.3333	,	55150	),
        147	:		(	31783.3333	,	56250	),
        148	:		(	31800	,	54716.6667	),
        149	:		(	31800	,	55333.3333	),
        150	:		(	31800	,	55700	),
        151	:		(	31800	,	56133.3333	),
        152	:		(	31816.6667	,	54666.6667	),
        153	:		(	31816.6667	,	55183.3333	),
        154	:		(	31816.6667	,	55216.6667	),
        155	:		(	31816.6667	,	55500	),
        156	:		(	31816.6667	,	55916.6667	),
        157	:		(	31816.6667	,	55966.6667	),
        158	:		(	31833.3333	,	54166.6667	),
        159	:		(	31833.3333	,	55766.6667	),
        160	:		(	31833.3333	,	55933.3333	),
        161	:		(	31833.3333	,	55983.3333	),
        162	:		(	31833.3333	,	57250	),
        163	:		(	31850	,	56000	),
        164	:		(	31866.6667	,	54200	),
        165	:		(	31866.6667	,	57283.3333	),
        166	:		(	31883.3333	,	55433.3333	),
        167	:		(	31883.3333	,	55966.6667	),
        168	:		(	31883.3333	,	56000	),
        169	:		(	31883.3333	,	57333.3333	),
        170	:		(	31883.3333	,	57483.3333	),
        171	:		(	31883.3333	,	57516.6667	),
        172	:		(	31900	,	54150	),
        173	:		(	31900	,	54783.3333	),
        174	:		(	31900	,	55133.3333	),
        175	:		(	31900	,	55466.6667	),
        176	:		(	31900	,	55883.3333	),
        177	:		(	31916.6667	,	54733.3333	),
        178	:		(	31916.6667	,	55933.3333	),
        179	:		(	31916.6667	,	56016.6667	),
        180	:		(	31916.6667	,	57250	),
        181	:		(	31916.6667	,	57716.6667	),
        182	:		(	31950	,	57883.3333	),
        183	:		(	31966.6667	,	55050	),
        184	:		(	31966.6667	,	57516.6667	),
        185	:		(	31983.3333	,	54166.6667	),
        186	:		(	31983.3333	,	55683.3333	),
        187	:		(	31983.3333	,	57416.6667	),
        188	:		(	32000	,	54133.3333	),
        189	:		(	32000	,	55483.3333	),
        190	:		(	32000	,	57166.6667	),
        191	:		(	32000	,	57300	),
        192	:		(	32000	,	57583.3333	),
        193	:		(	32016.6667	,	54200	),
        194	:		(	32016.6667	,	55066.6667	),
        195	:		(	32016.6667	,	57333.3333	),
        196	:		(	32016.6667	,	57466.6667	),
        197	:		(	32033.3333	,	54183.3333	),
        198	:		(	32033.3333	,	54566.6667	),
        199	:		(	32033.3333	,	55666.6667	),
        200	:		(	32033.3333	,	55783.3333	),
        201	:		(	32050	,	54633.3333	),
        202	:		(	32050	,	54866.6667	),
        203	:		(	32050	,	55383.3333	),
        204	:		(	32066.6667	,	53766.6667	),
        205	:		(	32066.6667	,	56350	),
        206	:		(	32083.3333	,	54250	),
        207	:		(	32083.3333	,	55650	),
        208	:		(	32083.3333	,	56866.6667	),
        209	:		(	32083.3333	,	57916.6667	),
        210	:		(	32100	,	53750	),
        211	:		(	32100	,	54866.6667	),
        212	:		(	32100	,	57416.6667	),
        213	:		(	32100	,	57816.6667	),
        214	:		(	32116.6667	,	54066.6667	),
        215	:		(	32116.6667	,	57566.6667	),
        216	:		(	32150	,	54200	),
        217	:		(	32150	,	54883.3333	),
        218	:		(	32150	,	54950	),
        219	:		(	32150	,	55050	),
        220	:		(	32150	,	56116.6667	),
        221	:		(	32166.6667	,	53750	),
        222	:		(	32166.6667	,	54116.6667	),
        223	:		(	32166.6667	,	55016.6667	),
        224	:		(	32166.6667	,	55950	),
        225	:		(	32183.3333	,	55033.3333	),
        226	:		(	32183.3333	,	55283.3333	),
        227	:		(	32183.3333	,	56066.6667	),
        228	:		(	32200	,	54066.6667	),
        229	:		(	32200	,	58000	),
        230	:		(	32216.6667	,	55600	),
        231	:		(	32216.6667	,	56016.6667	),
        232	:		(	32233.3333	,	54983.3333	),
        233	:		(	32233.3333	,	55733.3333	),
        234	:		(	32233.3333	,	56500	),
        235	:		(	32250	,	55750	),
        236	:		(	32250	,	56516.6667	),
        237	:		(	32250	,	57300	),
        238	:		(	32266.6667	,	54916.6667	),
        239	:		(	32266.6667	,	55900	),
        240	:		(	32283.3333	,	58050	),
        241	:		(	32300	,	55333.3333	),
        242	:		(	32321.3889	,	58075.5556	),
        243	:		(	32333.3333	,	55883.3333	),
        244	:		(	32333.3333	,	55966.6667	),
        245	:		(	32333.3333	,	57050	),
        246	:		(	32333.3333	,	57800	),
        247	:		(	32333.3333	,	57933.3333	),
        248	:		(	32333.3333	,	58133.3333	),
        249	:		(	32350	,	53833.3333	),
        250	:		(	32350	,	54133.3333	),
        251	:		(	32350	,	55833.3333	),
        252	:		(	32350	,	56550	),
        253	:		(	32350	,	57200	),
        254	:		(	32366.6667	,	54183.3333	),
        255	:		(	32366.6667	,	54766.6667	),
        256	:		(	32366.6667	,	57850	),
        257	:		(	32366.6667	,	57983.3333	),
        258	:		(	32383.3333	,	54350	),
        259	:		(	32383.3333	,	54666.6667	),
        260	:		(	32383.3333	,	55066.6667	),
        261	:		(	32383.3333	,	56683.3333	),
        262	:		(	32383.3333	,	57500	),
        263	:		(	32383.3333	,	57600	),
        264	:		(	32400	,	55350	),
        265	:		(	32400	,	56716.6667	),
        266	:		(	32400	,	56900	),
        267	:		(	32416.6667	,	56166.6667	),
        268	:		(	32416.6667	,	56333.3333	),
        269	:		(	32416.6667	,	57383.3333	),
        270	:		(	32416.6667	,	57583.3333	),
        271	:		(	32416.6667	,	58150	),
        272	:		(	32433.3333	,	56350	),
        273	:		(	32450	,	55033.3333	),
        274	:		(	32450	,	56033.3333	),
        275	:		(	32466.6667	,	54316.6667	),
        276	:		(	32466.6667	,	55233.3333	),
        277	:		(	32466.6667	,	56166.6667	),
        278	:		(	32483.3333	,	53516.6667	),
        279	:		(	32483.3333	,	53716.6667	),
        280	:		(	32483.3333	,	56350	),
        281	:		(	32483.3333	,	57750	),
        282	:		(	32500	,	54016.6667	),
        283	:		(	32500	,	54416.6667	),
        284	:		(	32500	,	56033.3333	),
        285	:		(	32516.6667	,	53483.3333	),
        286	:		(	32516.6667	,	54533.3333	),
        287	:		(	32516.6667	,	54583.3333	),
        288	:		(	32516.6667	,	54683.3333	),
        289	:		(	32516.6667	,	57750	),
        290	:		(	32533.3333	,	54566.6667	),
        291	:		(	32533.3333	,	55500	),
        292	:		(	32533.3333	,	57133.3333	),
        293	:		(	32566.6667	,	53416.6667	),
        294	:		(	32566.6667	,	56883.3333	),
        295	:		(	32566.6667	,	57366.6667	),
        296	:		(	32566.6667	,	57483.3333	),
        297	:		(	32583.3333	,	53383.3333	),
        298	:		(	32600	,	55583.3333	),
        299	:		(	32616.6667	,	54600	),
        300	:		(	32616.6667	,	55833.3333	),
        301	:		(	32616.6667	,	56483.3333	),
        302	:		(	32633.3333	,	53300	),
        303	:		(	32633.3333	,	53900	),
        304	:		(	32633.3333	,	55116.6667	),
        305	:		(	32633.3333	,	56350	),
        306	:		(	32633.3333	,	56750	),
        307	:		(	32650	,	54250	),
        308	:		(	32650	,	54450	),
        309	:		(	32666.6667	,	53616.6667	),
        310	:		(	32666.6667	,	55450	),
        311	:		(	32683.3333	,	54633.3333	),
        312	:		(	32683.3333	,	56883.3333	),
        313	:		(	32683.3333	,	57633.3333	),
        314	:		(	32683.3333	,	58133.3333	),
        315	:		(	32700	,	53966.6667	),
        316	:		(	32716.6667	,	55600	),
        317	:		(	32716.6667	,	57466.6667	),
        318	:		(	32750	,	53483.3333	),
        319	:		(	32750	,	53733.3333	),
        320	:		(	32750	,	54400	),
        321	:		(	32750	,	55650	),
        322	:		(	32750	,	57750	),
        323	:		(	32766.6667	,	55433.3333	),
        324	:		(	32766.6667	,	55633.3333	),
        325	:		(	32766.6667	,	57783.3333	),
        326	:		(	32766.6667	,	57883.3333	),
        327	:		(	32783.3333	,	53833.3333	),
        328	:		(	32783.3333	,	55900	),
        329	:		(	32800	,	55800	),
        330	:		(	32800	,	56416.6667	),
        331	:		(	32800	,	57050	),
        332	:		(	32816.6667	,	56516.6667	),
        333	:		(	32833.3333	,	54766.6667	),
        334	:		(	32833.3333	,	55483.3333	),
        335	:		(	32833.3333	,	56183.3333	),
        336	:		(	32833.3333	,	56433.3333	),
        337	:		(	32833.3333	,	57716.6667	),
        338	:		(	32850	,	57683.3333	),
        339	:		(	32866.6667	,	55916.6667	),
        340	:		(	32866.6667	,	56466.6667	),
        341	:		(	32883.3333	,	55416.6667	),
        342	:		(	32883.3333	,	55516.6667	),
        343	:		(	32883.3333	,	56800	),
        344	:		(	32900	,	55233.3333	),
        345	:		(	32900	,	56083.3333	),
        346	:		(	32900	,	56133.3333	),
        347	:		(	32916.6667	,	54966.6667	),
        348	:		(	32916.6667	,	57733.3333	),
        349	:		(	32933.3333	,	53950	),
        350	:		(	32933.3333	,	54266.6667	),
        351	:		(	32933.3333	,	55516.6667	),
        352	:		(	32933.3333	,	57116.6667	),
        353	:		(	32933.3333	,	57650	),
        354	:		(	32950	,	54216.6667	),
        355	:		(	32966.6667	,	55400	),
        356	:		(	32983.3333	,	54583.3333	),
        357	:		(	32983.3333	,	58050	),
        358	:		(	33000	,	53633.3333	),
        359	:		(	33000	,	55133.3333	),
        360	:		(	33000	,	55783.3333	),
        361	:		(	33016.6667	,	53700	),
        362	:		(	33016.6667	,	53950	),
        363	:		(	33016.6667	,	55050	),
        364	:		(	33016.6667	,	56483.3333	),
        365	:		(	33033.3333	,	57000	),
        366	:		(	33050	,	54066.6667	),
        367	:		(	33050	,	54150	),
        368	:		(	33050	,	56483.3333	),
        369	:		(	33066.6667	,	55850	),
        370	:		(	33083.3333	,	55966.6667	),
        371	:		(	33083.3333	,	56183.3333	),
        372	:		(	33083.3333	,	57433.3333	),
        373	:		(	33100	,	55133.3333	),
        374	:		(	33100	,	57633.3333	),
        375	:		(	33100	,	57733.3333	),
        376	:		(	33116.6667	,	54816.6667	),
        377	:		(	33116.6667	,	54833.3333	),
        378	:		(	33116.6667	,	55983.3333	),
        379	:		(	33132.5	,	58295.5556	),
        380	:		(	33133.3333	,	54933.3333	),
        381	:		(	33133.3333	,	57150	),
        382	:		(	33150	,	54150	),
        383	:		(	33166.6667	,	54966.6667	),
        384	:		(	33166.6667	,	57583.3333	),
        385	:		(	33183.3333	,	53900	),
        386	:		(	33183.3333	,	55700	),
        387	:		(	33183.3333	,	57466.6667	),
        388	:		(	33200	,	53800	),
        389	:		(	33200	,	54750	),
        390	:		(	33200	,	56783.3333	),
        391	:		(	33216.6667	,	54383.3333	),
        392	:		(	33216.6667	,	56683.3333	),
        393	:		(	33233.3333	,	53850	),
        394	:		(	33233.3333	,	54383.3333	),
        395	:		(	33233.3333	,	57133.3333	),
        396	:		(	33250	,	53900	),
        397	:		(	33250	,	53916.6667	),
        398	:		(	33250	,	54016.6667	),
        399	:		(	33250	,	54333.3333	),
        400	:		(	33250	,	56016.6667	),
        401	:		(	33250	,	57333.3333	),
        402	:		(	33255.8333	,	58019.1667	),
        403	:		(	33266.6667	,	53783.3333	),
        404	:		(	33266.6667	,	54416.6667	),
        405	:		(	33266.6667	,	55100	),
        406	:		(	33266.6667	,	55116.6667	),
        407	:		(	33283.3333	,	53933.3333	),
        408	:		(	33300	,	56350	),
        409	:		(	33333.3333	,	54783.3333	),
        410	:		(	33333.3333	,	56466.6667	),
        411	:		(	33333.3333	,	56933.3333	),
        412	:		(	33333.3333	,	57950	),
        413	:		(	33350	,	55616.6667	),
        414	:		(	33350	,	55633.3333	),
        415	:		(	33366.6667	,	53666.6667	),
        416	:		(	33366.6667	,	56483.3333	),
        417	:		(	33383.3333	,	55083.3333	),
        418	:		(	33383.3333	,	56416.6667	),
        419	:		(	33383.3333	,	57550	),
        420	:		(	33400	,	54683.3333	),
        421	:		(	33400	,	58316.6667	),
        422	:		(	33413.0556	,	56500.5556	),
        423	:		(	33416.6667	,	55633.3333	),
        424	:		(	33416.6667	,	57066.6667	),
        425	:		(	33450	,	54533.3333	),
        426	:		(	33450	,	57733.3333	),
        427	:		(	33466.6667	,	54200	),
        428	:		(	33466.6667	,	54650	),
        429	:		(	33466.6667	,	54666.6667	),
        430	:		(	33466.6667	,	55116.6667	),
        431	:		(	33466.6667	,	55200	),
        432	:		(	33466.6667	,	55616.6667	),
        433	:		(	33483.3333	,	55150	),
        434	:		(	33483.3333	,	55566.6667	),
        435	:		(	33483.3333	,	56150	),
        436	:		(	33483.3333	,	56483.3333	),
        437	:		(	33500	,	55216.6667	),
        438	:		(	33500	,	56750	),
        439	:		(	33500	,	56866.6667	),
        440	:		(	33516.6667	,	53783.3333	),
        441	:		(	33516.6667	,	54916.6667	),
        442	:		(	33516.6667	,	56400	),
        443	:		(	33516.6667	,	57833.3333	),
        444	:		(	33533.3333	,	54433.3333	),
        445	:		(	33533.3333	,	54600	),
        446	:		(	33533.3333	,	56900	),
        447	:		(	33533.3333	,	56933.3333	),
        448	:		(	33538.8889	,	56888.6111	),
        449	:		(	33544.1667	,	58197.2222	),
        450	:		(	33550	,	55666.6667	),
        451	:		(	33550	,	56900	),
        452	:		(	33566.6667	,	53850	),
        453	:		(	33583.3333	,	57300	),
        454	:		(	33583.3333	,	57583.3333	),
        455	:		(	33583.3333	,	58083.3333	),
        456	:		(	33600	,	54316.6667	),
        457	:		(	33600	,	55333.3333	),
        458	:		(	33600	,	56350	),
        459	:		(	33600	,	57133.3333	),
        460	:		(	33600	,	57983.3333	),
        461	:		(	33616.6667	,	53716.6667	),
        462	:		(	33616.6667	,	57633.3333	),
        463	:		(	33616.6667	,	58283.3333	),
        464	:		(	33650	,	53800	),
        465	:		(	33650	,	57533.3333	),
        466	:		(	33666.6667	,	54200	),
        467	:		(	33683.3333	,	53450	),
        468	:		(	33683.3333	,	53550	),
        469	:		(	33683.3333	,	54650	),
        470	:		(	33683.3333	,	57566.6667	),
        471	:		(	33716.6667	,	54533.3333	),
        472	:		(	33716.6667	,	58083.3333	),
        473	:		(	33716.6667	,	58266.6667	),
        474	:		(	33733.3333	,	54750	),
        475	:		(	33733.3333	,	56333.3333	),
        476	:		(	33733.3333	,	57833.3333	),
        477	:		(	33750	,	55283.3333	),
        478	:		(	33750	,	56466.6667	),
        479	:		(	33750	,	57483.3333	),
        480	:		(	33750	,	57616.6667	),
        481	:		(	33766.6667	,	58350	),
        482	:		(	33800	,	54000	),
        483	:		(	33800	,	58250	),
        484	:		(	33816.6667	,	55550	),
        485	:		(	33816.6667	,	57550	),
        486	:		(	33833.3333	,	56300	),
        487	:		(	33850	,	55166.6667	),
        488	:		(	33850	,	56883.3333	),
        489	:		(	33850	,	57016.6667	),
        490	:		(	33850	,	57683.3333	),
        491	:		(	33866.6667	,	55550	),
        492	:		(	33866.6667	,	57000	),
        493	:		(	33866.6667	,	57516.6667	),
        494	:		(	33866.6667	,	57616.6667	),
        495	:		(	33883.3333	,	53500	),
        496	:		(	33883.3333	,	54716.6667	),
        497	:		(	33883.3333	,	56450	),
        498	:		(	33883.3333	,	57383.3333	),
        499	:		(	33883.3333	,	57400	),
        500	:		(	33883.3333	,	57666.6667	),
        501	:		(	33883.3333	,	58416.6667	),
        502	:		(	33900	,	55150	),
        503	:		(	33900	,	56833.3333	),
        504	:		(	33916.6667	,	57783.3333	),
        505	:		(	33933.3333	,	55266.6667	),
        506	:		(	33933.3333	,	55683.3333	),
        507	:		(	33933.3333	,	56250	),
        508	:		(	33933.3333	,	57466.6667	),
        509	:		(	33933.3333	,	58166.6667	),
        510	:		(	33950	,	55850	),
        511	:		(	33950	,	56750	),
        512	:		(	33950	,	57900	),
        513	:		(	33950	,	58233.3333	),
        514	:		(	33966.6667	,	57100	),
        515	:		(	33966.6667	,	58283.3333	),
        516	:		(	33983.3333	,	53983.3333	),
        517	:		(	33983.3333	,	57900	),
        518	:		(	33983.3333	,	58150	),
        519	:		(	33983.3333	,	58250	),
        520	:		(	33989.1667	,	58285.5556	),
        521	:		(	34000	,	55666.6667	),
        522	:		(	34000	,	58250	),
        523	:		(	34016.6667	,	57650	),
        524	:		(	34033.3333	,	54283.3333	),
        525	:		(	34033.3333	,	55650	),
        526	:		(	34033.3333	,	57833.3333	),
        527	:		(	34033.3333	,	58133.3333	),
        528	:		(	34050	,	54783.3333	),
        529	:		(	34050	,	55883.3333	),
        530	:		(	34050	,	56200	),
        531	:		(	34066.6667	,	53933.3333	),
        532	:		(	34066.6667	,	54500	),
        533	:		(	34066.6667	,	56300	),
        534	:		(	34066.6667	,	56383.3333	),
        535	:		(	34066.6667	,	57350	),
        536	:		(	34066.6667	,	57716.6667	),
        537	:		(	34083.3333	,	57383.3333	),
        538	:		(	34095.5556	,	56214.1667	),
        539	:		(	34100	,	58000	),
        540	:		(	34116.6667	,	53950	),
        541	:		(	34133.3333	,	56400	),
        542	:		(	34150	,	56950	),
        543	:		(	34152.2222	,	58012.5	),
        544	:		(	34153.0556	,	58176.6667	),
        545	:		(	34166.6667	,	53833.3333	),
        546	:		(	34166.6667	,	53850	),
        547	:		(	34166.6667	,	54666.6667	),
        548	:		(	34166.6667	,	56683.3333	),
        549	:		(	34183.3333	,	53783.3333	),
        550	:		(	34183.3333	,	55733.3333	),
        551	:		(	34183.3333	,	57933.3333	),
        552	:		(	34189.1667	,	56339.4444	),
        553	:		(	34200	,	53933.3333	),
        554	:		(	34200	,	54750	),
        555	:		(	34200	,	56216.6667	),
        556	:		(	34200	,	56983.3333	),
        557	:		(	34200	,	57100	),
        558	:		(	34200	,	57783.3333	),
        559	:		(	34200	,	57866.6667	),
        560	:		(	34216.6667	,	57000	),
        561	:		(	34233.3333	,	55733.3333	),
        562	:		(	34233.3333	,	56866.6667	),
        563	:		(	34250	,	55933.3333	),
        564	:		(	34250	,	56683.3333	),
        565	:		(	34266.6667	,	53933.3333	),
        566	:		(	34266.6667	,	54316.6667	),
        567	:		(	34266.6667	,	57450	),
        568	:		(	34283.3333	,	54983.3333	),
        569	:		(	34283.3333	,	55233.3333	),
        570	:		(	34283.3333	,	56216.6667	),
        571	:		(	34283.3333	,	56916.6667	),
        572	:		(	34283.3333	,	57616.6667	),
        573	:		(	34290.5556	,	56388.8889	),
        574	:		(	34300	,	55966.6667	),
        575	:		(	34300	,	57233.3333	),
        576	:		(	34300	,	57733.3333	),
        577	:		(	34316.6667	,	56800	),
        578	:		(	34316.6667	,	57000	),
        579	:		(	34316.6667	,	57216.6667	),
        580	:		(	34316.6667	,	57350	),
        581	:		(	34319.1667	,	56805.2778	),
        582	:		(	34333.3333	,	55450	),
        583	:		(	34333.3333	,	55650	),
        584	:		(	34333.3333	,	57233.3333	),
        585	:		(	34333.3333	,	57716.6667	),
        586	:		(	34337.5	,	56713.6111	),
        587	:		(	34350	,	55766.6667	),
        588	:		(	34350	,	56650	),
        589	:		(	34350	,	57300	),
        590	:		(	34350	,	57433.3333	),
        591	:		(	34350	,	57850	),
        592	:		(	34351.6667	,	56398.3333	),
        593	:		(	34366.6667	,	53766.6667	),
        594	:		(	34366.6667	,	53833.3333	),
        595	:		(	34366.6667	,	54066.6667	),
        596	:		(	34366.6667	,	55166.6667	),
        597	:		(	34366.6667	,	55516.6667	),
        598	:		(	34366.6667	,	55783.3333	),
        599	:		(	34366.6667	,	56400	),
        600	:		(	34366.6667	,	57050	),
        601	:		(	34366.6667	,	57066.6667	),
        602	:		(	34366.6667	,	57566.6667	),
        603	:		(	34370	,	55225	),
        604	:		(	34382.7778	,	56541.6667	),
        605	:		(	34383.3333	,	56283.3333	),
        606	:		(	34383.3333	,	56383.3333	),
        607	:		(	34383.3333	,	56766.6667	),
        608	:		(	34400	,	53783.3333	),
        609	:		(	34400	,	56333.3333	),
        610	:		(	34400	,	56533.3333	),
        611	:		(	34400	,	57166.6667	),
        612	:		(	34400	,	57333.3333	),
        613	:		(	34400	,	57666.6667	),
        614	:		(	34400	,	57883.3333	),
        615	:		(	34411.6667	,	56402.2222	),
        616	:		(	34416.6667	,	56466.6667	),
        617	:		(	34416.6667	,	57283.3333	),
        618	:		(	34416.6667	,	57733.3333	),
        619	:		(	34419.7222	,	56422.5	),
        620	:		(	34433.3333	,	55983.3333	),
        621	:		(	34433.3333	,	57416.6667	),
        622	:		(	34433.3333	,	57716.6667	),
        623	:		(	34433.3333	,	57883.3333	),
        624	:		(	34442.2222	,	56443.6111	),
        625	:		(	34449.7222	,	56079.1667	),
        626	:		(	34450	,	55783.3333	),
        627	:		(	34450	,	56166.6667	),
        628	:		(	34450	,	56250	),
        629	:		(	34450	,	56283.3333	),
        630	:		(	34450	,	57600	),
        631	:		(	34453.3333	,	56390.5556	),
        632	:		(	34466.6667	,	57716.6667	),
        633	:		(	34466.6667	,	57850	),
        634	:		(	34483.3333	,	54333.3333	),
        635	:		(	34483.3333	,	55633.3333	),
        636	:		(	34483.3333	,	55650	),
        637	:		(	34483.3333	,	56850	),
        638	:		(	34497.5	,	56037.2222	),
        639	:		(	34500	,	55583.3333	),
        640	:		(	34500	,	56383.3333	),
        641	:		(	34516.6667	,	55250	),
        642	:		(	34519.7222	,	56798.8889	),
        643	:		(	34521.9444	,	56393.6111	),
        644	:		(	34522.7778	,	56277.7778	),
        645	:		(	34533.3333	,	56300	),
        646	:		(	34533.3333	,	56316.6667	),
        647	:		(	34550	,	55883.3333	),
        648	:		(	34566.6667	,	55700	),
        649	:		(	34583.3333	,	55833.3333	),
        650	:		(	34583.8889	,	56699.4444	),
        651	:		(	34589.4444	,	56252.7778	),
        652	:		(	34600	,	54550	),
        653	:		(	34600	,	55483.3333	),
        654	:		(	34600	,	56133.3333	),
        655	:		(	34605	,	56356.3889	),
        656	:		(	34633.3333	,	55616.6667	),
        657	:		(	34633.3333	,	55966.6667	),
        658	:		(	34633.3333	,	56333.3333	),
        659	:		(	34633.3333	,	56619.1667	),
        660	:		(	34646.9444	,	56062.7778	),
        661	:		(	34650	,	55233.3333	),
        662	:		(	34650	,	55866.6667	),
        663	:		(	34650	,	56600	),
        664	:		(	34650	,	56716.6667	),
        665	:		(	34650	,	56750	),
        666	:		(	34665	,	56219.4444	),
        667	:		(	34666.6667	,	54166.6667	),
        668	:		(	34666.6667	,	55266.6667	),
        669	:		(	34666.6667	,	56000	),
        670	:		(	34683.3333	,	55683.3333	),
        671	:		(	34683.3333	,	56733.3333	),
        672	:		(	34698.3333	,	56217.5	),
        673	:		(	34700	,	55900	),
        674	:		(	34715.2778	,	56073.3333	),
        675	:		(	34716.6667	,	55416.6667	),
        676	:		(	34716.6667	,	55533.3333	),
        677	:		(	34716.6667	,	55950	),
        678	:		(	34726.3889	,	56220	),
        679	:		(	34733.3333	,	55266.6667	),
        680	:		(	34733.3333	,	55716.6667	),
        681	:		(	34733.6111	,	56036.6667	),
        682	:		(	34742.2222	,	56098.3333	),
        683	:		(	34750	,	55333.3333	),
        684	:		(	34750	,	55683.3333	),
        685	:		(	34761.6667	,	56223.6111	),
        686	:		(	34762.5	,	56020.5556	),
        687	:		(	34763.3333	,	56385.2778	),
        688	:		(	34764.1667	,	56213.8889	),
        689	:		(	34766.6667	,	55750	),
        690	:		(	34781.1111	,	56053.3333	),
        691	:		(	34790.2778	,	56350	),
        692	:		(	34800	,	54916.6667	),
        693	:		(	34800	,	55233.3333	),
        694	:		(	34800	,	55366.6667	),
        695	:		(	34800	,	56216.6667	),
        696	:		(	34800	,	56233.3333	),
        697	:		(	34801.6667	,	56334.1667	),
        698	:		(	34803.0556	,	56085.8333	),
        699	:		(	34813.0556	,	56044.7222	),
        700	:		(	34816.6667	,	56100	),
        701	:		(	34816.6667	,	56183.3333	),
        702	:		(	34828.6111	,	56139.4444	),
        703	:		(	34833.3333	,	56116.6667	),
        704	:		(	34833.3333	,	56216.6667	),
        705	:		(	34833.3333	,	56233.3333	),
        706	:		(	34850	,	56083.3333	),
        707	:		(	34850	,	56116.6667	),
        708	:		(	34850	,	56216.6667	),
        709	:		(	34850	,	56233.3333	),
        710	:		(	34858.0556	,	56170.8333	),
        711	:		(	34860.2778	,	56052.2222	),
        712	:		(	34866.6667	,	54866.6667	),
        713	:		(	34866.6667	,	56166.6667	),
        714	:		(	34870.5556	,	56133.6111	),
        715	:		(	34875	,	56258.8889	),
        716	:		(	34877.2222	,	56029.7222	),
        717	:		(	34883.3333	,	56150	),
        718	:		(	34883.3333	,	56183.3333	),
        719	:		(	34883.3333	,	56200	),
        720	:		(	34883.3333	,	56216.6667	),
        721	:		(	34883.3333	,	56233.3333	),
        722	:		(	34885.2778	,	56060.5556	),
        723	:		(	34900	,	54950	),
        724	:		(	34900	,	55283.3333	),
        725	:		(	34900	,	56100	),
        726	:		(	34900	,	56133.3333	),
        727	:		(	34900	,	56150	),
        728	:		(	34900	,	56183.3333	),
        729	:		(	34905	,	56168.3333	),
        730	:		(	34905.2778	,	56194.4444	),
        731	:		(	34908.3333	,	56208.3333	),
        732	:		(	34909.7222	,	56152.2222	),
        733	:		(	34922.7778	,	56159.7222	),
        734	:		(	34966.6667	,	54950	),
    }

    D = []
    for _, target_coordinates in cities_coordinates.items():
        distances = []
        for _, coordinates in cities_coordinates.copy().items():
            distances.append(get_euclidean_distance(target_coordinates, coordinates))
        D.append(distances)

    home = 0
    max_iterations = 1000000
    cities = list(cities_coordinates.keys())
    city_indexes = [index - 1 for index in cities]

    state = get_random_solution(D, home, city_indexes, 100)
    print("-- Initial state solution --")
    print(cities[home], end="")
    for i in range(0, len(state.route)):
        print(" -> " + str(cities[state.route[i]]), end="")
    print(" -> " + str(cities[home]), end="")
    print("\n\nTotal distance: {0} miles".format(state.distance))
    print()

    state = hill_climbing(D, home, state, max_iterations, 0.1)
    print("-- Hill climbing solution --")
    print(cities[home], end="")
    for i in range(0, len(state.route)):
        print(" -> " + str(cities[state.route[i]]), end="")
    print(" -> " + str(cities[home]), end="")
    print("\n\nTotal distance: {0} miles".format(state.distance))
    print()


if __name__ == "__main__":
    main()

-- Initial state solution --
1 -> 358 -> 194 -> 392 -> 257 -> 324 -> 146 -> 143 -> 73 -> 642 -> 487 -> 317 -> 148 -> 382 -> 272 -> 714 -> 214 -> 533 -> 304 -> 515 -> 125 -> 655 -> 641 -> 610 -> 239 -> 393 -> 27 -> 707 -> 315 -> 675 -> 583 -> 734 -> 719 -> 293 -> 727 -> 537 -> 366 -> 555 -> 470 -> 676 -> 147 -> 222 -> 643 -> 616 -> 277 -> 296 -> 364 -> 433 -> 238 -> 435 -> 82 -> 722 -> 680 -> 573 -> 158 -> 294 -> 228 -> 376 -> 266 -> 530 -> 93 -> 411 -> 21 -> 630 -> 587 -> 195 -> 513 -> 606 -> 572 -> 192 -> 308 -> 440 -> 646 -> 729 -> 51 -> 188 -> 490 -> 599 -> 174 -> 482 -> 344 -> 335 -> 410 -> 466 -> 236 -> 578 -> 671 -> 115 -> 186 -> 78 -> 472 -> 565 -> 104 -> 320 -> 459 -> 614 -> 342 -> 354 -> 394 -> 97 -> 159 -> 276 -> 461 -> 589 -> 723 -> 372 -> 237 -> 4 -> 254 -> 248 -> 567 -> 645 -> 86 -> 258 -> 275 -> 580 -> 305 -> 493 -> 678 -> 639 -> 702 -> 531 -> 52 -> 453 -> 145 -> 255 -> 89 -> 44 -> 62 -> 190 -> 232 -> 595 -> 454 -> 351 -> 545 -> 76 -> 11 -> 489 -> 658 -> 400 -> 23 -> 119 